<a href="https://colab.research.google.com/github/AguaClara/Floc_pH/blob/master/NaOH_to_add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from aguaclara.core.units import unit_registry as u
import aguaclara.research.environmental_processes_analysis as epa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

"""define molecular weights"""
m_O = 15.999*u.g/u.mol
m_Na = 22.99*u.g/u.mol
m_H = 1.008*u.g/u.mol
m_NaOH = m_Na+m_O+m_H

"""Raw water characteristics"""
pH_0 = .5*(6.451398+6.704374)
ANC_0 = ((.5*(3.104+3.028)) * u.mg/u.L/m_NaOH).to(u.mmol/u.L)

def total_carbonates_closed(pH, ANC):
    """This function calculates total carbonates for a closed system given pH and ANC

    Parameters
    ----------
    pH : float
        pH of the sample
    ANC: float
        acid neutralizing capacity of the sample
    Returns
    -------
    The total carbonates of the sample
    Examples
    --------
    >>> total_carbonates_closed(1*u.mmol/u.L,8)
    1.017 mole/liter
    """
    return (ANC - epa.Kw/epa.invpH(pH) + epa.invpH(pH)) / (epa.alpha1_carbonate(pH) + 2 * epa.alpha2_carbonate(pH))


CT_0 = total_carbonates_closed(pH_0,ANC_0)

def pH_adjust(pH_0,ANC_0,Pi_base,Pi_CO3,Pi_Al,C_Al,pH_target):
  """This function calculates the required base (or acid) to adjust the pH to a target value. The buffering capacity is assumed to be completely due to carbonate species. The initial carbonate concentration is calculated based on the initial pH and the initial ANC.

  Parameters
  ----------
  pH_0: float
      pH of the sample
  ANC_0: float
      acid neutralizing capacity (Alkalinity) of the sample in eq/L.
  Pi_base: float
    equivalents of ANC per mole of base (or acid)
  Pi_CO3: float
    mole of carbonate per mole of base (or acid)
  Pi_Al : float
    equivalents of ANC per mole of aluminum coagulant
  C_Al
    concentration of aluminum coagulant in moles/L
  pH_target: float
    pH goal
  Returns
  -------
  The required concentration of base (or acid) in millimoles/L
  Examples
  --------
  >>> pH_adjust(5.91,0.2*u.mmol/u.L,1,1,0,0,7)
  2.2892822041250924 millimole/liter
  >>> pH_adjust(7,0.2*u.mmol/u.L,1,1,0,0,0,0,7)
  0.0 millimole/liter
  >>> pH_adjust(7,0*u.mmol/u.L,1,0,-3,1*u.mmol/u.L,7)
  3.0 millimole/liter
  """
  CT_0 = total_carbonates_closed(pH_0,ANC_0)
  B_num = CT_0 * (epa.alpha1_carbonate(pH_target) + 2 * epa.alpha2_carbonate(pH_target)) + epa.Kw/epa.invpH(pH_target) - epa.invpH(pH_target) - ANC_0 - Pi_Al*C_Al
  B_den = Pi_base - Pi_CO3*(epa.alpha1_carbonate(pH_target) + 2 * epa.alpha2_carbonate(pH_target))
  return (B_num/B_den).to(u.mmol/u.L)

  """target pH"""
pH_targets = np.arange(1,10,dtype=float)

Pi_base_NaOH = 1
Pi_CO3_NaOH = 0

C_NaOHs = [pH_adjust(pH_0,ANC_0,Pi_base_NaOH,Pi_CO3_NaOH,0,0,pH_target) for pH_target in pH_targets]

"""Display results in a pandas table"""
target = np.asarray(pH_targets,dtype=str)
myindex = ["[mmoles/L]","[mg/L]"]
row1 = [C_NaOH.magnitude for C_NaOH in C_NaOHs]
row2 = [(C_NaOH*m_NaOH).to(u.mg/u.L).magnitude for C_NaOH in C_NaOHs]
df = pd.DataFrame([row1,row2],index=myindex,columns=target)
print(df.round(3))

# Maybe take the negative value of mmol/L NaOH --> add that many moles of HCl to get acidic pHs

              1.0    2.0    3.0    4.0    5.0    6.0   7.0   8.0   9.0
[mmoles/L] -100.1 -10.08 -1.077 -0.176 -0.082  -0.04 0.024 0.047 0.064
[mg/L]     -4,003   -403 -43.06 -7.045 -3.262 -1.617  0.97 1.866 2.566
